Système de recmomendation très fortement corrélé avec [cette vidéo](https://www.youtube.com/watch?v=UoQyGG9mbSQ) 


In [ ]:
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [1]:
title_ratings = pd.read_csv("../movies_dataset/title.ratings.tsv", sep='\t', index_col='tconst')
title_ratings.head()

NameError: name 'pd' is not defined